In [ ]:
pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.2 MB/s eta 0:00:00


In [ ]:
import fitz
import cv2
import numpy as np
import matplotlib.pyplot as plt

def identify_stamps(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, threshold_image = cv2.threshold(gray_image, 0, 50, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(threshold_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    potential_stamps = []
    for contour in contours:
        area = cv2.contourArea(contour)

        if area > 600:
            x, y, w, h = cv2.boundingRect(contour)
            potential_stamps.append((x, y, w, h))

    return potential_stamps

def identify_stamps_in_pdf(pdf_path):

    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()

        img_np = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, pix.n))
        potential_stamps = identify_stamps(img_np)

        for (x, y, w, h) in potential_stamps:
            cv2.rectangle(img_np, (x, y), (x + w, y + h), (0, 500, 0), 2)

        plt.imshow(cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()


pdf_path = '/content/625-611_PIN for translation .pdf'
identify_stamps_in_pdf(pdf_path)
